In [1]:
import pandas as pd

real_X = pd.read_pickle("../data/lbnl/Unsequenced_real.pkl")
sequenced_real = pd.read_pickle("../data/lbnl/sequenced_real.pkl")
simulated_X = pd.read_pickle("../data/lbnl/Unsequenced_simulated.pkl")
sequenced_simulated = pd.read_pickle("../data/lbnl/Sequenced_simulated.pkl")
simulated_y = pd.read_pickle("../data/lbnl/simulated_y.pkl")
real_y= pd.read_pickle("../data/lbnl/real_y.pkl")
simulated_data = pd.read_pickle("../data/lbnl/simulated_data.pkl")
real_data = pd.read_pickle("../data/lbnl/real_data.pkl")

In [2]:
num_timestep = 10

## Feature Representation

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten

# Define model architecture
model = Sequential()
model.add(LSTM(50, input_shape=(num_timestep, sequenced_simulated.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on simulated data
model.fit(sequenced_simulated, simulated_y[num_timestep:], epochs=30, batch_size=32, validation_split=0.2)


Epoch 1/30


KeyboardInterrupt: 

## Few-Shot Learning Setup

In [ ]:
from tensorflow.keras.models import Model

# Extracting features using the LSTM model
feature_extractor = Model(inputs=model.input, outputs=model.layers[-3].output)

sequenced_simulated_features = feature_extractor.predict(sequenced_simulated)
sequenced_real_features = feature_extractor.predict(sequenced_real)


581/581 [==============================] - 2s 4ms/step


## Few-Shot Learning Model

In [ ]:
from tensorflow.keras.layers import Input, Subtract, Lambda
import tensorflow.keras.backend as K
import numpy as np

# Siamese network
input_shape = sequenced_simulated_features.shape[1:]

input_anchor = Input(shape=input_shape)
input_positive = Input(shape=input_shape)

merged_vector = Subtract()([input_anchor, input_positive])
distance = Lambda(lambda x: K.sqrt(K.sum(K.square(x), axis=1, keepdims=True)))(merged_vector)

siamese_network = Model(inputs=[input_anchor, input_positive], outputs=distance)

siamese_network.compile(optimizer='adam', loss='mse')

## Training the Few-Shot Model

In [ ]:
def sequence_labels(y, ids, timestep=10):
    sequenced_labels = []
    
    id_array = np.array(ids)
    
    for i in range(timestep, len(id_array)):
        if id_array[i-timestep] == id_array[i]:
            sequenced_labels.append(y[i])
            
    return np.array(sequenced_labels)

sequenced_real_y = sequence_labels(real_y, real_data['ID'], timestep=num_timestep)


## evaluate

In [ ]:
sequenced_real_y_1d = pd.read_pickle("../data/lbnl/sequenced_real_y_1d.pkl")

In [ ]:
def get_predictions_bulk(siamese_model, test_samples, known_anomalies, threshold):
    """
    Get predictions for each test sample based on distances to known anomalies.
    If the minimum distance of a sample to all known anomalies is less than the threshold,
    classify it as an anomaly (1), otherwise as normal (0).
    """
    # Reshape test samples and known anomalies to enable broadcasting
    test_samples_exp = test_samples[:, np.newaxis]
    known_anomalies_exp = known_anomalies[np.newaxis, :]
    
    # Calculate distances for each test sample against all known anomalies
    # Note: Here you'd ideally have a distance function that matches the one your Siamese network is trained on.
    #       If it's a simple Euclidean distance, the following will suffice:
    distances = np.sqrt(np.sum(np.square(test_samples_exp - known_anomalies_exp), axis=-1))
    
    # Find the minimum distance of each test sample to all known anomalies
    min_distances = distances.min(axis=1)
    
    # Return predictions based on the threshold
    return (min_distances < threshold).astype(int)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_k_shot_learning(k):
    
    def create_pairs(positive_samples, negative_samples):
        pairs = []
        labels = []
    
        # Pair every positive sample with every other positive sample and label as 0 (similar)
        for i in range(len(positive_samples)):
            for j in range(i+1, len(positive_samples)):
                pairs.append([positive_samples[i], positive_samples[j]])
                labels.append(0)
    
        # Pair every positive sample with every negative sample and label as 1 (different)
        for i in range(len(positive_samples)):
            for j in range(len(negative_samples)):
                pairs.append([positive_samples[i], negative_samples[j]])
                labels.append(1)
                
        return np.array(pairs), np.array(labels)

    # Get k known anomalies
    known_anomalies = sequenced_real_features[sequenced_real_y == 1][:k]
    
    # Create pairs
    pairs, labels = create_pairs(known_anomalies, sequenced_simulated_features)

    # Train the Siamese network
    siamese_network.fit([pairs[:, 0], pairs[:, 1]], labels, epochs=10, batch_size=32)

    # Predict distances
    train_distances = siamese_network.predict([pairs[:, 0], pairs[:, 1]])
    threshold = np.percentile(train_distances, 97)
    real_predict_y = get_predictions_bulk(siamese_network, sequenced_real_features, known_anomalies, threshold)

    # Evaluation
    conf_matrix = confusion_matrix(sequenced_real_y_1d, real_predict_y)
    accuracy = accuracy_score(sequenced_real_y_1d, real_predict_y)
    precision, recall, fscore, support = precision_recall_fscore_support(sequenced_real_y_1d, real_predict_y)
    
    results = {
        "k=":k, 
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": fscore,
        "confusion_matrix": conf_matrix
    }
    
    return results

In [ ]:
result_list = []
k_values = [5, 10, 50, 100]
for k in k_values:
    print(f"\nFor k = {k}")
    result_list.append(evaluate_k_shot_learning(k))


For k = 5
Epoch 1/10
5810/5810 [==============================] - 4s 722us/step - loss: 85.6526
Epoch 2/10
5810/5810 [==============================] - 4s 696us/step - loss: 85.6530
Epoch 3/10
5810/5810 [==============================] - 4s 729us/step - loss: 85.6528
Epoch 4/10
5810/5810 [==============================] - 4s 679us/step - loss: 85.6528
Epoch 5/10
5810/5810 [==============================] - 4s 690us/step - loss: 85.6529
Epoch 6/10
5810/5810 [==============================] - 4s 741us/step - loss: 85.6529
Epoch 7/10
5810/5810 [==============================] - 4s 691us/step - loss: 85.6528
Epoch 8/10
5810/5810 [==============================] - 4s 739us/step - loss: 85.6527
Epoch 9/10
5810/5810 [==============================] - 5s 779us/step - loss: 85.6529
Epoch 10/10
5810/5810 [==============================] - 4s 756us/step

For k = 10
Epoch 1/10
11621/11621 [==============================] - 587s 50ms/step - loss: 85.6864
Epoch 2/10
11621/11621 [===================

MemoryError: Unable to allocate 6.93 GiB for an array with shape (3722950, 500) and data type float32

In [ ]:
result_list

[{'k=': 5,
  'accuracy': 0.1560516406670253,
  'precision': array([1.        , 0.15418621]),
  'recall': array([0.00260648, 1.        ]),
  'f1_score': array([0.00519942, 0.26717736]),
  'confusion_matrix': array([[   41, 15689],
         [    0,  2860]], dtype=int64)},
 {'k=': 10,
  'accuracy': 0.1560516406670253,
  'precision': array([1.        , 0.15418621]),
  'recall': array([0.00260648, 1.        ]),
  'f1_score': array([0.00519942, 0.26717736]),
  'confusion_matrix': array([[   41, 15689],
         [    0,  2860]], dtype=int64)},
 {'k=': 50,
  'accuracy': 0.15578267885960193,
  'precision': array([1.        , 0.15414466]),
  'recall': array([0.00228862, 1.        ]),
  'f1_score': array([0.00456679, 0.26711497]),
  'confusion_matrix': array([[   36, 15694],
         [    0,  2860]], dtype=int64)}]